# Image Feature Extraction
- Corner detection, Blob detection, Texture features
- Real examples: Object recognition, Image matching

In [1]:
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
print('Feature extraction module loaded')

Feature extraction module loaded


## Image Moments
**Definition**: Weighted average of pixel intensities

**Uses**:
- Center of mass (centroid)
- Area
- Orientation
- Shape descriptors

In [2]:
# Create shape
y, x = np.ogrid[:200, :300]
img = ((y-100)**2/30**2 + (x-150)**2/50**2 < 1).astype(float)

# Calculate moments
m00 = img.sum()  # Area
m10 = (img * x).sum()
m01 = (img * y).sum()

# Centroid
cx = m10 / m00
cy = m01 / m00

print(f'Image moments:')
print(f'  Area (m00): {m00:.0f} pixels')
print(f'  Centroid: ({cx:.1f}, {cy:.1f})')

Image moments:
  Area (m00): 4689 pixels
  Centroid: (150.0, 100.0)


## Center of Mass
Find geometric center of objects
Useful for: tracking, alignment, robotics

In [3]:
# Using scipy function
com = ndimage.center_of_mass(img)
print(f'Center of mass: {com}')
print(f'Matches centroid calculation: {np.allclose([cy, cx], com)}')

Center of mass: (np.float64(100.0), np.float64(150.0))
Matches centroid calculation: True


## Label Connected Components
Identify separate objects in binary image
Each object gets unique label

In [4]:
# Multiple objects
multi_obj = np.zeros((200, 300), dtype=bool)
multi_obj[(y-60)**2 + (x-80)**2 < 30**2] = True
multi_obj[(y-140)**2 + (x-220)**2 < 25**2] = True
multi_obj[(y-100)**2 + (x-150)**2 < 20**2] = True

# Label
labeled, num_features = ndimage.label(multi_obj)
print(f'Connected components: {num_features} objects detected')

# Properties of each object
for i in range(1, num_features + 1):
    obj_mask = labeled == i
    area = obj_mask.sum()
    com = ndimage.center_of_mass(obj_mask)
    print(f'  Object {i}: Area={area} pixels, Center={com}')

Connected components: 3 objects detected
  Object 1: Area=2809 pixels, Center=(np.float64(60.0), np.float64(80.0))
  Object 2: Area=1245 pixels, Center=(np.float64(100.0), np.float64(150.0))
  Object 3: Area=1941 pixels, Center=(np.float64(140.0), np.float64(220.0))


## Real Example: Cell Counting
Count cells in microscopy image
Measure size distribution

In [5]:
# Simulate cell image
np.random.seed(42)
cells = np.zeros((400, 400), dtype=bool)
n_cells = 30

for _ in range(n_cells):
    cy = np.random.randint(50, 350)
    cx = np.random.randint(50, 350)
    radius = np.random.randint(10, 25)
    y, x = np.ogrid[:400, :400]
    mask = (y-cy)**2 + (x-cx)**2 < radius**2
    cells[mask] = True

# Separate touching cells (watershed-like)
cells = ndimage.binary_opening(cells, iterations=2)

# Label and count
labeled_cells, n_cells_detected = ndimage.label(cells)
print(f'Cell counting results:')
print(f'  Total cells: {n_cells_detected}')

# Measure properties
areas = []
for i in range(1, n_cells_detected + 1):
    cell_mask = labeled_cells == i
    area = cell_mask.sum()
    areas.append(area)

areas = np.array(areas)
print(f'  Average cell size: {areas.mean():.1f} ± {areas.std():.1f} pixels')
print(f'  Size range: [{areas.min()}, {areas.max()}] pixels')

Cell counting results:
  Total cells: 19
  Average cell size: 1345.0 ± 1359.9 pixels
  Size range: [301, 6007] pixels


## Texture Features
**Local Binary Patterns (LBP)**: Compare pixel with neighbors
**Histogram of Oriented Gradients (HOG)**: Edge orientations

Useful for: classification, recognition

In [6]:
# Simple texture measure: Gradient variance
img_texture = np.random.rand(100, 100)
sobel_x = ndimage.sobel(img_texture, axis=1)
sobel_y = ndimage.sobel(img_texture, axis=0)
gradient_mag = np.hypot(sobel_x, sobel_y)

texture_variance = gradient_mag.var()
print(f'Texture feature:')
print(f'  Gradient variance: {texture_variance:.4f}')
print(f'  High variance = rough texture')
print(f'  Low variance = smooth texture')

Texture feature:
  Gradient variance: 0.3995
  High variance = rough texture
  Low variance = smooth texture


## Summary

### Feature Extraction Functions:
```python
# Moments and centroid
com = ndimage.center_of_mass(img)

# Connected components
labeled, n_objects = ndimage.label(binary_img)

# Object properties
areas = ndimage.sum(binary_img, labeled, range(1, n_objects+1))
centroids = ndimage.center_of_mass(img, labeled, range(1, n_objects+1))
```

### Applications:
✓ **Microscopy**: Cell counting, size measurement  
✓ **Medical**: Tumor detection, organ segmentation  
✓ **Industrial**: Defect detection, parts counting  
✓ **Robotics**: Object localization, grasping  
✓ **Astronomy**: Star detection, galaxy classification  